In [ ]:
import pandas as pd
import openpyxl
import numpy as np

In [ ]:
dff = pd.read_excel("Summer Camp Task Data.xlsx" , engine='openpyxl')
df = dff.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111083 entries, 0 to 111082
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   action                   111083 non-null  object 
 1   created_at               111083 non-null  int64  
 2   source_event_id          111083 non-null  object 
 3   device_id                109596 non-null  object 
 4   post_page_offset         35287 non-null   float64
 5   tokens                   35287 non-null   object 
 6   post_index_in_post_list  75796 non-null   float64
 7   post_token               75795 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 4.7+ MB


In [ ]:
lpp = df[df.action != "click_post"]
cp = df[df.action == "click_post"]
exm1 = lpp[lpp.post_page_offset == 137]
exm2 = lpp[lpp.post_page_offset == 138]
exm2.shape[0] - exm1.shape[0]

1

In [ ]:
first_load = lpp[lpp.post_page_offset == 0]
first_load["token_length"] = (first_load['tokens'].str.len())/10
tl_under10 = first_load[first_load.token_length < 10]
dark_query = tl_under10.shape[0]/first_load.shape[0]
dark_query_percent = dark_query*100
dark_query_percent

c:\users\mazinet rayaneh\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


13.156440022111665

In [ ]:
df.sort_values("source_event_id" , ascending=False)
query_lpp = lpp.source_event_id
query_cp = cp.source_event_id
dif_sei_lpp = query_lpp.drop_duplicates(keep='first', inplace=False)
dif_sei_cpp = query_cp.drop_duplicates(keep='first', inplace=False)
dif_sei_lpp.sort_values(ascending=False)
dif_sei_cpp.sort_values(ascending=False)
common_sei = len(np.intersect1d(dif_sei_cpp , dif_sei_lpp))
only_lpp_sei= dif_sei_lpp.shape[0] - common_sei
query_bounce_rate=only_lpp_sei/dif_sei_lpp.shape[0]
query_bounce_rate

0.3705361901716235

In [ ]:
def token_extractor(df,i):
    l=[]
    for item in df.loc[i,"tokens"].split(","):
        l.append(item[1:])
    l[-1]=l[-1][0:8]
    idk=[]
    for item in l[0:3]:
        idk.append(item)
    idk.append(df.loc[i,"source_event_id"])
    return idk
token_extractor(lpp,4)

['wXvjJTIc', 'wXvb5_SX', 'wXvb54Gs', '6c390fce-34b6-4e19-837e-6bc295a72514']

In [ ]:
lpp_sorted = lpp.sort_values("source_event_id" , ascending=False)
first_load = lpp[lpp.post_page_offset == 0]
l=[]
for i in range(first_load.shape[0]):
    l.append(token_extractor(first_load , first_load.index[i]))
lol=pd.DataFrame(l)
lol

,0,1,2,3
0,wXvHbNqU,wXvDrof9,wXv_aXHk,479905f2-3bb4-4c90-8e88-5ce4b60d0a73
1,wXvb7cpv,wXvH3hL0,wXvjmDaE,b9b609bd-a486-42fb-b1a9-5ca03cf4a215
2,wXv_oVL4,wXvrIudl,wXknzwdX,3898ebc8-ae35-4e6f-b924-5ce64f764dd1
3,wXr_DuyQ,wXpr43e5,wXl_rC8t,576278e1-9af0-4ed9-95c8-09a6b6c2e958
4,wXvra2va,wXvr6ZFP,wXvrKWRu,18e1e48c-0d83-476d-a374-fff86ac50966
...,...,...,...,...
9040,wXvD5mUV,wXv_o5Cb,wXv3HWXX,731df5bf-0a68-4526-bcd8-6e5b4414b282
9041,wXvTRHJH,wXuHdYVk,wXu_BWDr,a5580ad0-fcd5-4f63-8299-ecf5d972c17d
9042,wXnzpKrs,wXfj3D6R,wXTfoqq7,9e2634d6-f7bc-4e68-8d77-119bece8141e
9043,wXv3qoPQ,wXvT7ylu,wXvnIJuH,f55b2133-4f95-4cc5-8d18-1b47bd3dca63


In [ ]:
li=[]
cp1=cp.copy()
cp2=cp1.drop_duplicates(subset=["post_token"])
fpc = cp2[cp2['source_event_id'].isin(lol[3]) & cp2['post_token'].isin(lol[0])]
spc = cp2[cp2['source_event_id'].isin(lol[3]) & cp2['post_token'].isin(lol[1])]
tpc = cp2[cp2['source_event_id'].isin(lol[3]) & cp2['post_token'].isin(lol[2])]
frames = [fpc , spc , tpc]
allcp = pd.concat(frames)
allcp_main =allcp.drop_duplicates(subset=["source_event_id"])
allcp_main.shape[0]/first_load.shape[0]


0.27993366500829187